In [2]:
import geopandas as gpd
import pandas as pd
import folium
import pydeck as pdk
import warnings
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import matplotlib as mpl
warnings.filterwarnings('ignore')

mpl.use('TkAgg')  # !IMPORTANT

def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [5]:
indo = gpd.read_file('FIX_INA_Pupuk.shp')
ax = indo.geometry.plot(figsize=(10,10), color='none', edgecolor='gainsboro', zorder=3)
# embed_map

<Axes: >

In [6]:
len(indo)

34

In [ ]:
GREAT_CIRCLE_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/flights.json"  # noqa

df_n = pd.read_json(GREAT_CIRCLE_LAYER_DATA)

# Use pandas to prepare data for tooltip
df_n["from_name"] = df_n["from"].apply(lambda f: f["name"])
df_n["to_name"] = df_n["to"].apply(lambda t: t["name"])

# Define a layer to display on a map
layer = pdk.Layer(
    "GreatCircleLayer",
    df_n,
    pickable=True,
    get_stroke_width=12,
    get_source_position="from.coordinates",
    get_target_position="to.coordinates",
    get_source_color=[64, 255, 0],
    get_target_color=[0, 128, 200],
    auto_highlight=True,
)

print(layer)

# Set the viewport location
view_state = pdk.ViewState(latitude=50, longitude=-40, zoom=1, bearing=0, pitch=0)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{from_name} to {to_name}"},
)
r.picking_radius = 10

r.to_html("great_circle_layer.html")

# df_n.iloc[0]['from']['coordinates']

In [ ]:
m = folium.Map(location=[50, -40], zoom_start=1)

for _, row in df.iterrows():
    from_coords = row["from"]["coordinates"]
    to_coords = row["to"]["coordinates"]
    from_name = row["from_name"]
    to_name = row["to_name"]

    folium.PolyLine(locations=[from_coords, to_coords],
                    color='blue',
                    weight=12,
                    tooltip=f'{from_name} to {to_name}').add_to(m)


m.save("great_circle_map_folium.html")

In [7]:
myl = list(indo.MEAN)
myl.sort()
print(myl)
med = len(myl) // 2
print("Median of Mean Data: {}".format(myl[med]))

[1.20508413669, 1.38054625314, 1.44404702931, 1.50355371583, 1.50389812964, 1.50595387766, 1.51236026076, 1.51467413353, 1.51576116908, 1.51706247814, 1.51707161853, 1.53202599956, 1.53222020453, 1.53515771432, 1.56428020236, 1.56512209308, 1.5754733483, 1.57652300024, 1.59237447767, 1.59986514131, 1.60138642408, 1.62038136136, 1.63154578265, 1.6432110722, 1.66180367047, 1.68508420633, 1.83200111759, 1.89718463245, 1.92323040858, 1.98236163249, 1.99016524099, 2.00364606463, 2.04198140907, 2.0522960072]
Median of Mean Data: 1.57652300024


In [8]:
pupuk, demand = list(indo.Dosis_Pupu), list(indo.Total_kebu)

pupuk_conv = [int(x.replace('.', '')) for x in pupuk]
indo.Dosis_Pupu = pupuk_conv
pupuk_conv.sort()
demand.sort()

print(pupuk_conv)
print(demand)
print("Median of Pupuk Hortikultura Data: {}".format(pupuk_conv[med]))
print("Median of Pupuk Perkebunan Data: {}".format(demand[med]))

[1376100, 2852190, 3563010, 3609540, 3935250, 5710320, 5931090, 6077610, 6609240, 7280460, 8437770, 8587026, 8743866, 9080280, 9529740, 9908910, 9931689, 10534590, 11451330, 11498850, 11748330, 11823618, 13888710, 21968415, 27683041, 29068380, 32414580, 35706330, 56579490, 61079286, 70496512, 142374323, 188520750, 221509530]
[0.0, 12.55, 15.975, 23.7, 28.325, 28.6, 29.95, 31.775, 38.25, 55.625, 57.9, 65.275, 71.9, 74.525, 75.65, 86.925, 91.7, 98.675, 99.3, 107.575, 133.375, 163.45, 174.075, 182.6, 186.875, 207.475, 228.275, 367.725, 411.6, 488.225, 539.75, 577.9, 659.925, 910.125]
Median of Pupuk Hortikultura Data: 10534590
Median of Pupuk Perkebunan Data: 98.675


In [9]:
a = set(indo.loc[(indo.MEAN < myl[med]) & 
     ((indo.Dosis_Pupu > pupuk_conv[med]) & 
     (indo.Total_kebu > demand[med]))].Provinsi)
print(list(a))
# tujuan prior

b = set(indo.loc[(indo.MEAN < myl[med]) & 
     ((indo.Dosis_Pupu > pupuk_conv[med]) |
     (indo.Total_kebu > demand[med]))].Provinsi)
# tujuan prior_2
c = list(b.difference(a))

print(c)

['Aceh', 'Sumatera Barat', 'Sumatera Utara', 'Kalimantan Barat', 'Bengkulu']
['Sumatera Selatan', 'Kalimantan Timur', 'Sulawesi Utara', 'Kalimantan Selatan', 'Sulawesi Barat', 'Kalimantan Tengah']


In [10]:
d = list(indo.loc[(indo.MEAN > myl[med]) & 
     ((indo.Dosis_Pupu > pupuk_conv[med]) | 
     (indo.Total_kebu > demand[med]))].Provinsi)
# distributhor
print(d)


['Riau', 'Sulawesi Selatan', 'Lampung', 'Jawa Tengah', 'Nusa Tenggara Barat', 'Jambi', 'Jawa Timur', 'Yogyakarta', 'Nusa Tenggara Timur', 'Jawa Barat', 'Sulawesi Tenggara']


In [11]:
lon1 = indo.iloc[0].geometry.centroid.x
lat1 = indo.iloc[0].geometry.centroid.y
lon2 = indo.iloc[1].geometry.centroid.x
lat2 = indo.iloc[1].geometry.centroid.y
print(lon1, lat1, lon2, lat2)

106.55274196400221 -2.44845864447203 122.37615124729896 0.6868471160930348


In [12]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [13]:
def stroke_factory(val):
    # print(val)
    if val <= 100:
        return 20
    elif val <= 500:
        return 17
    elif val <= 1000:
        return 14
    elif val <= 2500:
        return 12
    elif val <= 5000:
        return 10
    elif val <= 10000:
        return 7
    else:
        return 4

In [14]:
prior1 = ['Sumatera Barat', 'Bengkulu', 'Aceh', 'Sumatera Utara', 'Kalimantan Barat']
prior2 = ['Kalimantan Tengah', 'Sulawesi Utara', 'Kalimantan Timur', 'Kalimantan Selatan', 'Sumatera Selatan', 'Sulawesi Barat']
distributor = ['Riau', 'Sulawesi Selatan', 'Lampung', 'Jawa Tengah', 'Nusa Tenggara Barat', 'Jambi', 'Jawa Timur', 'Yogyakarta', 'Nusa Tenggara Timur', 'Jawa Barat', 'Sulawesi Tenggara']

# layer[i]1 -> distributor[i] to prior1
# layer[i]2 -> distributor[i] to prior2

# df structures 
# from, from_coords, to, to_coords

list_of_gdf1 = []
list_of_gdf2 = []
df = None
points = 100

# Creating curved coordinates using sine and cosine functions
curve_factor = 0.5 
t = np.linspace(0, 1, points)

for i in range(len(distributor)):
    from_x = float(indo.loc[indo.Provinsi == distributor[i]].geometry.centroid.x)
    from_y = float(indo.loc[indo.Provinsi == distributor[i]].geometry.centroid.y)
                    
    to_coords_1, distances_1, stroke_1, c_lat_1, c_lon_1 = [], [], [], [], []
    for j in range(len(prior1)):
        lng = float(indo.loc[indo.Provinsi == prior1[j]].geometry.centroid.x)
        lat = float(indo.loc[indo.Provinsi == prior1[j]].geometry.centroid.y)
        to_coords_1.append([lng,lat])
        distances_1.append(int(haversine(from_y, from_x, lat, lng)))
        stroke_1.append(stroke_factory(distances_1[j]))
        mid_lat = (from_y + lat) / 2
        mid_lon = (from_x + lng) / 2
        c_lat_1.append(curve_factor * np.sin(t * np.pi) + mid_lat)
        c_lon_1.append(curve_factor * np.cos(t * np.pi) + mid_lon)

    list_of_gdf1.append(pd.DataFrame({
        'from' : distributor[i],
        'from_x' : from_x,
        'from_y' : from_y,
        'to' : prior1,
        'to_coords' : to_coords_1,
        'distance(km)' : distances_1,
        'stroke_val' : stroke_1,
        # 'curve_lat' : c_lat_1,
        # 'curve_lon' : c_lon_1
    }))

    to_coords_2, distances_2, stroke_2, c_lat_2, c_lon_2= [], [], [], [], []
    for j in range(len(prior2)):
        lng = float(indo.loc[indo.Provinsi == prior2[j]].geometry.centroid.x)
        lat = float(indo.loc[indo.Provinsi == prior2[j]].geometry.centroid.y)
        to_coords_2.append([lng,lat])
        distances_2.append(int(haversine(from_y, from_x, lat, lng)))
        stroke_2.append(stroke_factory(distances_2[j]))
        mid_lat = (from_y + lat) / 2
        mid_lon = (from_x + lng) / 2
        c_lat_2.append(curve_factor * np.sin(t * np.pi) + mid_lat)
        c_lon_2.append(curve_factor * np.cos(t * np.pi) + mid_lon)


    list_of_gdf2.append(pd.DataFrame({
        'from' : distributor[i],
        'from_x' : from_x,
        'from_y' : from_y,
        'to' : prior2,
        'to_coords' : to_coords_2,
        'distance(km)' : distances_2,
        'stroke_val' : stroke_2,
        # 'curve_lat' : c_lat_2,
        # 'curve_lon' : c_lon_2
    }))

print((list_of_gdf1[0]))
# for i in list_of_gdf2:
#     print(i)

   from      from_x   from_y                to  \
0  Riau  101.808173  0.50988    Sumatera Barat   
1  Riau  101.808173  0.50988          Bengkulu   
2  Riau  101.808173  0.50988              Aceh   
3  Riau  101.808173  0.50988    Sumatera Utara   
4  Riau  101.808173  0.50988  Kalimantan Barat   

                                    to_coords  distance(km)  stroke_val  
0   [100.46672122302289, -0.8517519864316823]           212          17  
1   [102.33714758212847, -3.5330712218439566]           453          17  
2      [96.91088682159325, 4.224480160407013]           683          14  
3     [99.05331159952856, 2.1908112168957947]           358          17  
4  [111.12096450369842, -0.08665338514863673]          1037          12  


In [20]:
# Get land coordinates
indo_provinces = pd.read_csv('daftar_nama_daerah.csv')
indo_provinces = indo_provinces.iloc[:33]
indo_provinces.head()

,nid,parent_nid,name,serial,type,latitude,longitude,status
0,1,0,Provinsi Aceh,11,1,4.695135,96.749399,1
1,2,0,Provinsi Sumatera Utara,12,1,2.115355,99.545097,1
2,3,0,Provinsi Sumatera Barat,13,1,-0.739940,100.800005,1
3,4,0,Provinsi Riau,14,1,0.293347,101.706829,1
4,5,0,Provinsi Jambi,15,1,-1.485183,102.438058,1


In [26]:
indo_provinces.iloc[0]['name']

'Provinsi Aceh'

In [34]:
# Get region 1 land coordinate
prior1 = ['Sumatera Barat', 'Bengkulu', 'Aceh', 'Sumatera Utara', 'Kalimantan Barat']
prior1_coords_lng, prior1_coords_lat = [], []
for province in prior1:
    for idx in range(len(indo_provinces)):
        # print(province, indo_provinces.iloc[idx]['name'])
        if province in indo_provinces.iloc[idx]['name']:
            prior1_coords_lng.append(indo_provinces.iloc[idx].longitude)
            prior1_coords_lat.append(indo_provinces.iloc[idx].latitude)

# print(prior1_coords_lng, prior1_coords_lat)

# Get region 2 land coordinate
prior2 = ['Kalimantan Tengah', 'Sulawesi Utara', 'Kalimantan Timur', 'Kalimantan Selatan', 'Sumatera Selatan', 'Sulawesi Barat']
prior2_coords_lng, prior2_coords_lat = [], []
for province in prior2:
    for idx in range(len(indo_provinces)):
        # print(province, indo_provinces.iloc[idx]['name'])
        if province in indo_provinces.iloc[idx]['name']:
            prior2_coords_lng.append(indo_provinces.iloc[idx].longitude)
            prior2_coords_lat.append(indo_provinces.iloc[idx].latitude)

# print(prior2_coords_lng, prior2_coords_lat)

# Get distributor region land coordinate
distributor = ['Riau', 'Sulawesi Selatan', 'Lampung', 'Jawa Tengah', 'Nusa Tenggara Barat', 'Jambi', 'Jawa Timur', 'Yogyakarta', 'Nusa Tenggara Timur', 'Jawa Barat', 'Sulawesi Tenggara']
from_x, from_y = [], []
for province in distributor:
    for idx in range(len(indo_provinces)):
        # print(province, indo_provinces.iloc[idx]['name'])
        if province in indo_provinces.iloc[idx]['name'] and indo_provinces.iloc[idx]['name'] != 'Provinsi Kepulauan Riau':
            from_x.append(indo_provinces.iloc[idx].longitude)
            from_y.append(indo_provinces.iloc[idx].latitude)

# print(from_x, len(from_y))


[100.8000051, 102.3463875, 96.7493993, 99.5450974, 111.4752851] [-0.7399397, -3.5778471, 4.695135, 2.1153547, -0.2787808]
[113.3823545, 123.9750018, 116.419389, 115.2837585, 103.914399, 119.2320784] [-1.6814878, 0.6246932, 1.6406296, -3.0926415, -3.3194374, -2.8441371]
[101.7068294, 119.9740534, 105.4068079, 110.1402594, 117.3616476, 102.4380581, 112.2384017, 110.4262088, 121.0793705, 107.668887, 122.174605] 11


In [39]:
print(prior1_coords_lat[0])
print(prior2_coords_lat[0])
print(from_x[0])

-0.7399397
-1.6814878
101.7068294


In [43]:
# prior1 = ['Sumatera Barat', 'Bengkulu', 'Aceh', 'Sumatera Utara', 'Kalimantan Barat']
# prior2 = ['Kalimantan Tengah', 'Sulawesi Utara', 'Kalimantan Timur', 'Kalimantan Selatan', 'Sumatera Selatan', 'Sulawesi Barat']
# distributor = ['Riau', 'Sulawesi Selatan', 'Lampung', 'Jawa Tengah', 'Nusa Tenggara Barat', 'Jambi', 'Jawa Timur', 'Yogyakarta', 'Nusa Tenggara Timur', 'Jawa Barat', 'Sulawesi Tenggara']

# layer[i]1 -> distributor[i] to prior1
# layer[i]2 -> distributor[i] to prior2

# df structures 
# from, from_coords, to, to_coords

list_of_gdf1 = []
list_of_gdf2 = []
df = None

for i in range(len(distributor)):
    # from_x = float(indo.loc[indo.Provinsi == distributor[i]].geometry.centroid.x)
    # from_y = float(indo.loc[indo.Provinsi == distributor[i]].geometry.centroid.y)
                    
    to_coords_1, distances_1, stroke_1 = [], [], []
    for j in range(len(prior1)):
        lng = prior1_coords_lng[j]
        lat = prior1_coords_lat[j]
        to_coords_1.append([lng,lat])
        distances_1.append(int(haversine(from_y[i], from_x[i], lat, lng)))
        stroke_1.append(stroke_factory(distances_1[j]))
    # print(distributor[i], len(from_coords), len(prior1), len(to_coords_1), to_coords_1)
    list_of_gdf1.append(pd.DataFrame({
        'from' : distributor[i],
        'from_x' : from_x[i],
        'from_y' : from_y[i],
        'to' : prior1,
        'to_coords' : to_coords_1,
        'distance(km)' : distances_1,
        'stroke_val' : stroke_1
    }))

    to_coords_2, distances_2, stroke_2= [], [], []
    for j in range(len(prior2)):
        lng = prior2_coords_lng[j]
        lat = prior2_coords_lat[j]
        to_coords_2.append([lng,lat])
        distances_2.append(int(haversine(from_y[i], from_x[i], lat, lng)))
        stroke_2.append(stroke_factory(distances_2[j]))
    # print(distributor[i], len(from_coords), len(prior1), len(to_coords_1), to_coords_1)
    list_of_gdf2.append(pd.DataFrame({
        'from' : distributor[i],
        'from_x' : from_x[i],
        'from_y' : from_y[j],
        'to' : prior2,
        'to_coords' : to_coords_2,
        'distance(km)' : distances_2,
        'stroke_val' : stroke_2
    }))

print((list_of_gdf1[8]))
print((list_of_gdf2[8]))
# for i in list_of_gdf2:
#     print(i)

                  from     from_x    from_y                to  \
0  Nusa Tenggara Timur  121.07937 -8.657382    Sumatera Barat   
1  Nusa Tenggara Timur  121.07937 -8.657382          Bengkulu   
2  Nusa Tenggara Timur  121.07937 -8.657382              Aceh   
3  Nusa Tenggara Timur  121.07937 -8.657382    Sumatera Utara   
4  Nusa Tenggara Timur  121.07937 -8.657382  Kalimantan Barat   

                   to_coords  distance(km)  stroke_val  
0  [100.8000051, -0.7399397]          2411          12  
1  [102.3463875, -3.5778471]          2146          12  
2     [96.7493993, 4.695135]          3079          10  
3    [99.5450974, 2.1153547]          2670          10  
4  [111.4752851, -0.2787808]          1414          12  
                  from     from_x    from_y                  to  \
0  Nusa Tenggara Timur  121.07937 -1.485183   Kalimantan Tengah   
1  Nusa Tenggara Timur  121.07937 -1.485183      Sulawesi Utara   
2  Nusa Tenggara Timur  121.07937 -1.485183    Kalimantan Timur   

In [18]:
list_of_gdf2[0]

,from,from_x,from_y,to,to_coords,distance(km),stroke_val
0,Riau,101.808173,0.50988,Kalimantan Tengah,"[113.41766550554112, -1.6023108940793072]",1311,12
1,Riau,101.808173,0.50988,Sulawesi Utara,"[124.52309102286353, 1.2620528185493833]",2526,10
2,Riau,101.808173,0.50988,Kalimantan Timur,"[116.47044691495688, 0.46004240363960425]",1630,12
3,Riau,101.808173,0.50988,Kalimantan Selatan,"[115.43955951552073, -2.9932866172095562]",1564,12
4,Riau,101.808173,0.50988,Sumatera Selatan,"[104.17037690270533, -3.2174332792568228]",490,17
5,Riau,101.808173,0.50988,Sulawesi Barat,"[119.34400751686911, -2.461705235447351]",1977,12


In [325]:
layers_1 = []

for df in list_of_gdf1:
    # print(df)
    layers_1.append(pdk.Layer(
        "GreatCircleLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_arc_source_position="[from_x, from_y]",
        get_arc_target_position="to_coords",
        # get_arc_center="[curve_lon, curve_lat]",
        get_arc_height=100000,
        get_source_color=[255,0,0],
        get_target_color=[255,122,0],
        auto_highlight=True
    ))
    print(layers_1)

layers_2 = []

for df in list_of_gdf2:
    layers_2.append(pdk.Layer(
        "GreatCircleLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_arc_source_position="[from_x, from_y]",
        get_arc_target_position="to_coords",
        # get_arc_center="[curve_lon, curve_lat]",
        get_source_color=[255,122,0],
        get_target_color=[112,255,0],
        get_arc_height=100000,
        auto_highlight=True
    ))

print(layers_1[0].id)

[{
  "@@type": "GreatCircleLayer",
  "autoHighlight": true,
  "data": [
    {
      "distance(km)": 212,
      "from": "Riau",
      "from_x": 101.80817323985067,
      "from_y": 0.509879523489547,
      "stroke_val": 17,
      "to": "Sumatera Barat",
      "to_coords": [
        100.46672122302289,
        -0.8517519864316823
      ]
    },
    {
      "distance(km)": 453,
      "from": "Riau",
      "from_x": 101.80817323985067,
      "from_y": 0.509879523489547,
      "stroke_val": 17,
      "to": "Bengkulu",
      "to_coords": [
        102.33714758212847,
        -3.5330712218439566
      ]
    },
    {
      "distance(km)": 683,
      "from": "Riau",
      "from_x": 101.80817323985067,
      "from_y": 0.509879523489547,
      "stroke_val": 14,
      "to": "Aceh",
      "to_coords": [
        96.91088682159325,
        4.224480160407013
      ]
    },
    {
      "distance(km)": 358,
      "from": "Riau",
      "from_x": 101.80817323985067,
      "from_y": 0.509879523489547,
     

In [331]:
view_state = pdk.ViewState(latitude=0.7893, longitude=113.9213, zoom=3, bearing=0, pitch=50)

r = pdk.Deck(
    layers= layers_2,
    initial_view_state=view_state,
    tooltip={"text": "{from} distribusi ke {to}"},
)
# print(r)
r.picking_radius = 10

r.to_html("prior2.html")

In [ ]:
view_state = pdk.ViewState(latitude=0.7893, longitude=113.9213, zoom=3, bearing=0, pitch=50)

r = pdk.Deck(
    layers= layers_1,
    initial_view_state=view_state,
    tooltip={"text": "{from} distribusi ke {to}"},
)
# print(r)
r.picking_radius = 10

r.to_html("prior2.html")

In [342]:
gdfs = list_of_gdf1
for el in list_of_gdf2:
    gdfs.append(el)
print(len(gdfs)) 

22


In [44]:
layers_3 = []

for df in list_of_gdf1:
    layers_3.append(pdk.Layer(
        "ArcLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_tilt=15,
        get_source_color=[255,0,0],
        get_target_color=[255,122,0],
        auto_highlight=True
    ))

layers_4 = []

for df in list_of_gdf2:
    layers_4.append(pdk.Layer(
        "ArcLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_tilt=15,
        get_source_color=[255,122,0],
        get_target_color=[112,255,0],
        auto_highlight=True
    ))

layers = layers_3
for el in layers_4:
    layers.append(el)

view_state = pdk.ViewState(latitude=0.7893, longitude=113.9213, zoom=4, bearing=20, pitch=100)

r = pdk.Deck(
    layers= layers,
    initial_view_state=view_state,
    tooltip={"text": "{from} distribusi ke {to}"},
)
# print(r)
r.picking_radius = 10

r.to_html('prior_arc.html')
